# Validation testing for Natural Language Question to SPARQL Query

Create a df for the lcquad queries

In [8]:
import pandas as pd

In [6]:
lcquad_path = "../../lcquad2.0.train.json"

In [7]:
!ls -l {lcquad_path}

-rw-r--r--@ 1 i857913  staff  26561843 Feb  6 16:58 ../../lcquad2.0.train.json


In [53]:
lcquad_df = pd.read_json(lcquad_path)

In [80]:
len(lcquad_df)

24180

Make a sparql runner

In [15]:
from wikibaseintegrator import wbi_helpers
from wikibaseintegrator.wbi_config import config as wbi_config
import logging

In [18]:
wbi_config['USER_AGENT'] = 'AskwikiBot/1.0 (https://www.wikidata.org/wiki/User:What_Tottles_Meant)'
wbi_config['BACKOFF_MAX_TRIES'] = 1


In [89]:
from requests.exceptions import HTTPError
def run_sparql(query):
    try:
        results = wbi_helpers.execute_sparql_query(query)
    except HTTPError as he:
        logging.error(f'HTTPError {he}')
        print(f"failed query {query}")
        return None
    # print(results)
    if 'boolean' in results:
        return pd.DataFrame([{'Boolean': results['boolean'] }])
    jsonResult = [dict([(k, b[k]['value']) for k in b]) for b in results['results']['bindings']]
    df = pd.DataFrame.from_dict(jsonResult)
    return df


In [90]:
def validate_queries(qs):
    validation_results = []
    count = 0
    for q in qs:
        # print(q)
        df = run_sparql(q)
        result_count = 0
        if df is None:
            run_result = 'Fail'
            print(f"Failed query number {count}")
        else:
            run_result = 'Pass'
            result_count = len(df)
        validation_results.append((run_result, result_count))
        count += 1
    return validation_results 

In [98]:
v = validate_queries(lcquad_df['sparql_wikidata'].loc[0:999])

ERROR:backoff:Giving up execute_sparql_query(...) after 1 tries (requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0AASK+WHERE+%7B+wd%3AQ30+wdt%3AP2997+%3Fobj+filter%28%3Fobj+%3D+t1410874016%29+%7D+&format=json)
ERROR:root:HTTPError 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0AASK+WHERE+%7B+wd%3AQ30+wdt%3AP2997+%3Fobj+filter%28%3Fobj+%3D+t1410874016%29+%7D+&format=json


failed query ASK WHERE { wd:Q30 wdt:P2997 ?obj filter(?obj = t1410874016) } 
Failed query number 588


In [100]:
v_df = pd.DataFrame(v)

In [101]:
len(v_df[v_df[0] == 'Pass'])

999

Now get a bunch of generated sparql

In [1]:
modelName = "jstilb/t5"

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

/opt/homebrew/anaconda3/envs/conda210/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained(modelName)
model = AutoModelForSeq2SeqLM.from_pretrained(modelName)
translator = pipeline("translation", tokenizer=tokenizer, model=model)


/opt/homebrew/anaconda3/envs/conda210/lib/python3.9/site-packages/transformers/pipelines/__init__.py:885: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


In [99]:
def generate_sparql(question):
    translation = translator(f"translate English to SPARQL: {question}")
    if translation is None or len(translation) == 0 or 'translation_text' not in translation[0]:
        return None
    sparql = translation[0]['translation_text']
    logging.info(f'sparql {sparql}')
    return sparql


In [102]:
generate_sparql('What are the most common types of liver infection')

/opt/homebrew/anaconda3/envs/conda210/lib/python3.9/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


"SELECT DISTINCT ?sbj ?sbj_label WHERE  ?sbj wdt:P31 wd:Q31 . ?sbj rdfs:label ?sbj_label . FILTER(STRSTARTS(lcase(?sbj_label), 'en')) "

In [129]:
def generate_lots_of_sparql(l):
    result = []
    count = 1
    for s in l:
        if count % 10 == 0:
            print(count)
        result.append(generate_sparql(s))
        count += 1
    print(count)
    return result


In [130]:
queries = generate_lots_of_sparql(lcquad_df['question'].loc[0:499])

cucu
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
501


In [131]:
len(queries)

500

In [132]:
v = validate_queries(queries)

ERROR:backoff:Giving up execute_sparql_query(...) after 1 tries (requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0Aselect+distinct+%3Fsbj+where++%3Fsbj+wdt%3AP31+wd%3AQ220+.+%3Fsbj+wdt%3AP31+wd%3AQ220+&format=json)
ERROR:root:HTTPError 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0Aselect+distinct+%3Fsbj+where++%3Fsbj+wdt%3AP31+wd%3AQ220+.+%3Fsbj+wdt%3AP31+wd%3AQ220+&format=json


failed query select distinct ?sbj where  ?sbj wdt:P31 wd:Q220 . ?sbj wdt:P31 wd:Q220 
Failed query number 0


ERROR:backoff:Giving up execute_sparql_query(...) after 1 tries (requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0Aselect+distinct+%3Fsbj+where++%3Fsbj+wdt%3AP31+wd%3AQ220+.+%3Fsbj+wdt%3AP31+wd%3AQ220+&format=json)
ERROR:root:HTTPError 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0Aselect+distinct+%3Fsbj+where++%3Fsbj+wdt%3AP31+wd%3AQ220+.+%3Fsbj+wdt%3AP31+wd%3AQ220+&format=json


failed query select distinct ?sbj where  ?sbj wdt:P31 wd:Q220 . ?sbj wdt:P31 wd:Q220 
Failed query number 1


ERROR:backoff:Giving up execute_sparql_query(...) after 1 tries (requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0ASELECT+%3Fanswer+WHERE++wd%3AQ220+wdt%3AP31+%3Fanswer+.+%3Fanswer+wdt%3AP31+wd%3AQ220+&format=json)
ERROR:root:HTTPError 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0ASELECT+%3Fanswer+WHERE++wd%3AQ220+wdt%3AP31+%3Fanswer+.+%3Fanswer+wdt%3AP31+wd%3AQ220+&format=json


...

failed query select distinct ?sbj where  ?sbj wdt:P31 wd:Q220 . ?sbj wdt:P31 wd:Q220 
Failed query number 495
failed query select distinct ?sbj where  ?sbj wdt:P31 wd:Q220 . ?sbj wdt:P31 wd:Q220 
Failed query number 496


ERROR:backoff:Giving up execute_sparql_query(...) after 1 tries (requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0Aselect+distinct+%3Fsbj+where++%3Fsbj+wdt%3AP31+wd%3AQ220+.+%3Fsbj+wdt%3AP31+wd%3AQ220+&format=json)
ERROR:root:HTTPError 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0Aselect+distinct+%3Fsbj+where++%3Fsbj+wdt%3AP31+wd%3AQ220+.+%3Fsbj+wdt%3AP31+wd%3AQ220+&format=json
ERROR:backoff:Giving up execute_sparql_query(...) after 1 tries (requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0Aselect+distinct+%3Fsbj+where++%3Fsbj+wdt%3AP31+wd%3AQ220+.+%3Fsbj+wdt%3AP31+wd%3AQ220+&format=json)
ERROR:root:HTTPError 400 Client Error: Bad Request

failed query select distinct ?sbj where  ?sbj wdt:P31 wd:Q220 . ?sbj wdt:P31 wd:Q220 
Failed query number 497
failed query select distinct ?sbj where  ?sbj wdt:P31 wd:Q220 . ?sbj wdt:P31 wd:Q220 
Failed query number 498


ERROR:backoff:Giving up execute_sparql_query(...) after 1 tries (requests.exceptions.HTTPError: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0ASELECT+DISTINCT+%3Fsbj+%3Fsbj_label+WHERE++%3Fsbj+wdt%3AP31+wd%3AQ31+.+%3Fsbj+rdfs%3Alabel+%3Fsbj_label+.+FILTER%28STRSTARTS%28lcase%28%3Fsbj_label%29%2C+%27t%27%29%29+.+FILTER+%28lang%28%3Fsbj_label%29+%3D+%27t%27%29++LIMIT+25&format=json)
ERROR:root:HTTPError 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql?query=%23Tool%3A+WikibaseIntegrator+wbi_functions.execute_sparql_query%0ASELECT+DISTINCT+%3Fsbj+%3Fsbj_label+WHERE++%3Fsbj+wdt%3AP31+wd%3AQ31+.+%3Fsbj+rdfs%3Alabel+%3Fsbj_label+.+FILTER%28STRSTARTS%28lcase%28%3Fsbj_label%29%2C+%27t%27%29%29+.+FILTER+%28lang%28%3Fsbj_label%29+%3D+%27t%27%29++LIMIT+25&format=json


failed query SELECT DISTINCT ?sbj ?sbj_label WHERE  ?sbj wdt:P31 wd:Q31 . ?sbj rdfs:label ?sbj_label . FILTER(STRSTARTS(lcase(?sbj_label), 't')) . FILTER (lang(?sbj_label) = 't')  LIMIT 25
Failed query number 499


In [133]:
len(v)


500

In [134]:
v_df = pd.DataFrame(v)

Sadly, none of the generated sparql queries actually succeed...

In [135]:
len(v_df[v_df[0] == 'Pass'])

0